In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import math
from torch.utils.data import Dataset, DataLoader
from sklearn import datasets
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.preprocessing import StandardScaler#para escalar caracteristicas
from sklearn.model_selection import train_test_split # separar mas facil la data de train y test
import pandas as pd
from sklearn.utils import shuffle
from ast import literal_eval
import time
import sys
import os
from scipy.signal import convolve2d
from functions.auxiliares import PaddingSameSize, MatrixFormat_To_Vector, RemoveOversizedMatrix, StandardSize_Padding, CNN_Features_Format, batch_format, KernelSize, StandardSize_InitialConvolution, StandardSize
import ast

In [2]:
df_raw = pd.read_csv('D:/.Memoria/Test/Datasets/ds_trainval_pred_filtered.csv', delimiter="ᶶ", engine='python', encoding='utf-8')

df_raw.columns

Index(['id', 'query', 'time', 'assign', 'bgp', 'distinct', 'extend', 'filter',
       'filter_bound', 'filter_contains', 'filter_eq', 'filter_exists',
       'filter_ge', 'filter_gt', 'filter_isBlank', 'filter_isIRI',
       'filter_isLiteral', 'filter_lang', 'filter_langMatches', 'filter_le',
       'filter_lt', 'filter_ne', 'filter_not', 'filter_notexists', 'filter_or',
       'filter_regex', 'filter_sameTerm', 'filter_str', 'filter_strends',
       'filter_strstarts', 'filter_subtract', 'graph', 'group', 'has_slice',
       'join', 'json_cardinality', 'leftjoin', 'max_slice_limit',
       'max_slice_start', 'minus', 'multi', 'notoneof', 'order', 'path*',
       'path+', 'path?', 'pathN*', 'pathN+', 'pcs0', 'pcs1', 'pcs10', 'pcs11',
       'pcs12', 'pcs13', 'pcs14', 'pcs15', 'pcs16', 'pcs17', 'pcs18', 'pcs19',
       'pcs2', 'pcs20', 'pcs21', 'pcs22', 'pcs23', 'pcs24', 'pcs3', 'pcs4',
       'pcs5', 'pcs6', 'pcs7', 'pcs8', 'pcs9', 'project', 'reduced',
       'sequence', 'slice', 'to

In [3]:
def p_lst(lst):
    print("+++++++++++++++++++++++TOTAL+++++++++++++++++++++++++")
    print(len(lst))
    print("")
    for x in lst:
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        print("+++++++++++++++++++++++++queries2_"+str(x)+"++++++++++++++++++++++++++++++++++++++")
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        print("-------triple-------")
        print(df_raw['triple'][x])
        print("-------bgp----------")
        print(df_raw['bgp'][x])
        print("------unions---------")
        print(df_raw['query'][x].count('UNION'))
        print("-----optionals--------")
        print(df_raw['query'][x].count('OPTIONAL'))
        print("-----selects------")
        print(df_raw['query'][x].count('SELECT'))
        print("------query--------")
        print(df_raw['query'][x])
        print("------trees---------")
        print(df_raw['trees'][x])

In [4]:
import unicodedata as ud

latin_letters= {}

def is_latin(uchr):
    try: return latin_letters[uchr]
    except KeyError:
         return latin_letters.setdefault(uchr, 'LATIN' in ud.name(uchr))
def only_roman_chars(unistr):
    return all(is_latin(uchr)
           for uchr in unistr
           if uchr.isalpha())

only_roman_chars(u"ελληνικά means greek")

False

In [5]:
a = []
for i in range(len(df_raw['bgp'])):
    if 'LITERAL' in df_raw['trees'][i] \
    and not only_roman_chars(df_raw['query'][i]):
        a.append(i)

p_lst(a)


+++++++++++++++++++++++TOTAL+++++++++++++++++++++++++
35

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++queries2_1215++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-------triple-------
1.0
-------bgp----------
1.0
------unions---------
0
-----optionals--------
0
-----selects------
1
------query--------
SELECT  ?var1 WHERE   { ?var1  <http://www.wikidata.org/prop/direct/P3906>  "שאנן שיא" } 
------trees---------
["VAR_URI_LITERALᶲhttp://www.wikidata.org/prop/direct/P3906"]
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++queries2_1920++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-------triple-------
1.0
-------bgp----------
1.0
------unions---------
0
-----optionals--------
0
-----selects------
1
------query--------
SELECT  * WHERE   { ?var1  ?var2  "Тубероза"@en } L

-------bgp----------
1.0
------unions---------
0
-----optionals--------
0
-----selects------
1
------query--------
SELECT  ?var1 ?var1Label ?var2 WHERE   { ?var1 <http://www.wikidata.org/prop/direct/P31>/(<http://www.wikidata.org/prop/direct/P279>)* <http://www.wikidata.org/entity/Q515> .     ?var1  ?var1Label  "Москва"@ru   } ORDER BY ASC(?var1Label) 
------trees---------
["JOIN", ["JOIN", ["VAR_URI_URIᶲhttp://www.wikidata.org/prop/direct/P279"], ["VAR_URI_VARᶲhttp://www.wikidata.org/prop/direct/P31"]], ["VAR_VAR_LITERALᶲNONE"]]
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++queries2_12244++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-------triple-------
2.0
-------bgp----------
1.0
------unions---------
0
-----optionals--------
0
-----selects------
1
------query--------
SELECT  * WHERE   { ?var1  ?var2Label  "اليمن"@ar ;            ?var3       ?var4   } 
------trees-------

+++++++++++++++++++++++++queries2_15067++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-------triple-------
4.0
-------bgp----------
1.0
------unions---------
0
-----optionals--------
0
-----selects------
1
------query--------
SELECT  ?var1 ?var1Label ?var2 WHERE   { ?var1  <http://www.wikidata.org/prop/direct/P31>  <http://www.wikidata.org/entity/Q515> ;            ?var1Label            "Москва"@ru ;            ?var1Label            "Омск"@ru ;            <http://www.wikidata.org/prop/direct/P625>  ?var2   } ORDER BY ASC(?var1Label) 
------trees---------
[ "JOINᶲhttp://www.wikidata.org/prop/direct/P31ᶲhttp://www.wikidata.org/prop/direct/P625" ,   [ "JOINᶲhttp://www.wikidata.org/prop/direct/P31ᶲhttp://www.wikidata.org/prop/direct/P625" ,     [ "JOINᶲhttp://www.wikidata.org/prop/direct/P31ᶲhttp://www.wikidata.org/prop/direct/P625" ,       [ "VAR_URI_URIᶲhttp://www.wikidata.org/prop/direct/P31" ] ,       [ "VAR_URI_VARᶲhttp://

------query--------
SELECT  * WHERE   { ?var1  <http://www.wikidata.org/prop/direct/P31>  <http://www.wikidata.org/entity/Q4167410> ;            <http://schema.org/description>  "Категорії"@uk   } 
------trees---------
[ "JOINᶲhttp://www.wikidata.org/prop/direct/P31ᶲhttp://schema.org/description" ,   [ "VAR_URI_URIᶲhttp://www.wikidata.org/prop/direct/P31" ] ,   [ "VAR_URI_LITERALᶲhttp://schema.org/description" ] ] 


In [6]:
lst = [0, 22, 275, 457, 556, 666, 766, 800, 998, 1248, 2303, 3222, 3599, 4732, 5095, 5429, 6985, 7788, 8358, 8830, 8930, 8932, 9568, 10757, 10969, 11000, 11725, 12008, 12345, 12664, 13456, 13963, 14233, 16790, 17865, 21291, 22004, 22088, 23391, 24200]
p_lst(lst)

+++++++++++++++++++++++TOTAL+++++++++++++++++++++++++
40

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++queries2_0++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-------triple-------
4.0
-------bgp----------
2.0
------unions---------
0
-----optionals--------
1
-----selects------
2
------query--------
SELECT  ?var1 ?var2 (SAMPLE(?var3) AS ?var4) WHERE   { { SELECT DISTINCT  ?var1 ?var2       WHERE         { ?var2  <http://www.wikidata.org/prop/statement/P355>  <http://www.wikidata.org/entity/Q15142> .           ?var1  <http://www.wikidata.org/prop/P355>  ?var2         }       LIMIT   101     }     OPTIONAL       { ?var2  ?var3                 ?var5 .         ?var6  <http://wikiba.se/ontology#qualifier>  ?var3       }   } GROUP BY ?var1 ?var2 
------trees---------
["LEFT_JOIN" , [ "JOINᶲhttp://www.wikidata.org/prop/statement/P355ᶲhttp://www.wikidata.org/prop/P355" ,   [ "VAR_UR

SELECT  ?var1 ?var2 (SAMPLE(?var3) AS ?var4) WHERE   { { SELECT DISTINCT  ?var1 ?var2       WHERE         { ?var2  <http://www.wikidata.org/prop/statement/P136>  <http://www.wikidata.org/entity/Q5185279> .           ?var1  <http://www.wikidata.org/prop/P136>  ?var2         }       LIMIT   101     }     OPTIONAL       { ?var2  ?var3                 ?var5 .         ?var6  <http://wikiba.se/ontology#qualifier>  ?var3       }   } GROUP BY ?var1 ?var2 
------trees---------
["LEFT_JOIN" , [ "JOINᶲhttp://www.wikidata.org/prop/statement/P136ᶲhttp://www.wikidata.org/prop/P136" ,   [ "VAR_URI_URIᶲhttp://www.wikidata.org/prop/statement/P136" ] ,   [ "VAR_URI_VARᶲhttp://www.wikidata.org/prop/P136" ] ] , [ "JOINᶲhttp://wikiba.se/ontology#qualifier" ,   [ "VAR_URI_VARᶲhttp://wikiba.se/ontology#qualifier" ] ,   [ "VAR_VAR_VARᶲNONE" ] ] ]
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++queries2_2303++++++++++++++++++++++++++++++++++++++
+++++++++++++++++

+++++++++++++++++++++++++queries2_5429++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-------triple-------
3.0
-------bgp----------
1.0
------unions---------
0
-----optionals--------
0
-----selects------
1
------query--------
SELECT DISTINCT  ?var1 ?var2 ?var3 WHERE   { ?var1  <http://www.wikidata.org/prop/direct/P31>  <http://www.wikidata.org/entity/Q81672> ;            ?var3                 <http://www.wikidata.org/entity/Q60> ;            <http://www.wikidata.org/prop/direct/P625>  ?var2   } 
------trees---------
[ "JOINᶲhttp://www.wikidata.org/prop/direct/P31ᶲhttp://www.wikidata.org/prop/direct/P625ᶲhttp://www.wikidata.org/entity/Q60" ,   [ "JOINᶲhttp://www.wikidata.org/prop/direct/P31ᶲhttp://www.wikidata.org/prop/direct/P625" ,     [ "VAR_URI_URIᶲhttp://www.wikidata.org/prop/direct/P31" ] ,     [ "VAR_URI_VARᶲhttp://www.wikidata.org/prop/direct/P625" ]   ] ,   [ "VAR_VAR_URIᶲhttp://www.wikidata.org/entity/Q60" ] ] 
++++

SELECT  ?var1 ?var1Label ?var2 ?var3 ?var4 ?var5 ?var6 ?var7 ?var8 ?var9 ?var10 ?var11 ?var12 ?var13 ?var14 ?var15 ?var16 WHERE   { ?var1  <http://www.wikidata.org/prop/direct/P31>  <http://www.wikidata.org/entity/Q179700> ;            <http://www.wikidata.org/prop/direct/P571>  ?var2 ;            <http://www.wikidata.org/prop/direct/P186>  ?var3 ;            <http://www.wikidata.org/prop/direct/P18>  ?var4 ;            <http://www.wikidata.org/prop/direct/P170>  ?var6     OPTIONAL       { ?var1  <http://www.wikidata.org/prop/direct/P10>  ?var5 }     OPTIONAL       { ?var1  <http://www.wikidata.org/prop/direct/P625>  ?var7 }     OPTIONAL       { ?var1  <http://www.wikidata.org/prop/direct/P276>  ?var8 }     OPTIONAL       { ?var1  <http://www.wikidata.org/prop/direct/P2048>  ?var9 }     OPTIONAL       { ?var1  <http://www.wikidata.org/prop/direct/P2049>  ?var10 }     OPTIONAL       { ?var1  <http://www.wikidata.org/prop/direct/P2067>  ?var11 }     OPTIONAL       { ?var1  <http://www.wi

SELECT DISTINCT  ?var1 ?var2 WHERE   {   { ?var1  <http://www.wikidata.org/prop/direct/P31>  <http://www.wikidata.org/entity/Q2881060> }     UNION       { ?var1  <http://www.wikidata.org/prop/direct/P31>  <http://www.wikidata.org/entity/Q4117139> }     UNION       { ?var1  <http://www.wikidata.org/prop/direct/P31>  <http://www.wikidata.org/entity/Q8513> }     UNION       { ?var1  <http://www.wikidata.org/prop/direct/P31>  <http://www.wikidata.org/entity/Q324254> }     OPTIONAL       { ?var1  <http://www.wikidata.org/prop/direct/P1687>  ?var2 }   } 
------trees---------
["LEFT_JOIN" , ["LEFT_JOIN" , ["LEFT_JOIN" , ["LEFT_JOIN" , ["VAR_URI_URIᶲhttp://www.wikidata.org/prop/direct/P31"] , ["VAR_URI_URIᶲhttp://www.wikidata.org/prop/direct/P31"]] , ["VAR_URI_URIᶲhttp://www.wikidata.org/prop/direct/P31"]] , ["VAR_URI_URIᶲhttp://www.wikidata.org/prop/direct/P31"]] , ["VAR_URI_VARᶲhttp://www.wikidata.org/prop/direct/P1687"]]
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

SELECT DISTINCT  ?var1 ?var2 ?var3 ?var4 ?var5 (COUNT(DISTINCT ?var6) AS ?var7) WHERE   {   { ?var8  <http://www.wikidata.org/prop/direct/P106>  <http://www.wikidata.org/entity/Q36180> }     UNION       { ?var8  <http://www.wikidata.org/prop/direct/P106>  <http://www.wikidata.org/entity/Q482980> }     UNION       { ?var8  <http://www.wikidata.org/prop/direct/P106>  <http://www.wikidata.org/entity/Q49757> }     ?var8  <http://www.wikidata.org/prop/direct/P27>  ?var9 ;            <http://www.wikidata.org/prop/direct/P268>  ?var3 .     ?var8 ((((<http://www.wikidata.org/prop/direct/P37>|<http://www.wikidata.org/prop/direct/P103>)|<http://www.wikidata.org/prop/direct/P364>)|<http://www.wikidata.org/prop/direct/P407>)|<http://www.wikidata.org/prop/direct/P1412>)|<http://www.wikidata.org/prop/direct/P2439> <http://www.wikidata.org/entity/Q150>       { ?var8  <http://www.wikidata.org/prop/direct/P21>  <http://www.wikidata.org/entity/Q6581072> }     UNION       { ?var8  <http://www.wikidata.or

+++++++++++++++++++++++++queries2_23391++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-------triple-------
4.0
-------bgp----------
4.0
------unions---------
1
-----optionals--------
1
-----selects------
1
------query--------
SELECT  ?var1 ?var1Label ?var2 WHERE   {   { ?var1  <http://www.wikidata.org/prop/direct/P19>  <http://www.wikidata.org/entity/Q22> }     UNION       { ?var1  <http://www.wikidata.org/prop/direct/P19>  ?var3 .         ?var3 (<http://www.wikidata.org/prop/direct/P131>)* <http://www.wikidata.org/entity/Q22>       }     ?var1  <http://www.wikidata.org/prop/direct/P31>  <http://www.wikidata.org/entity/Q5>     OPTIONAL       { ?var1  <http://www.wikidata.org/prop/direct/P569>  ?var2 }   } ORDER BY DESC(?var1Label) OFFSET  0 LIMIT   4000 
------trees---------
["LEFT_JOIN" , ["JOIN", ["LEFT_JOIN" , ["VAR_URI_URIᶲhttp://www.wikidata.org/prop/direct/P19"] , ["JOIN", ["VAR_URI_VARᶲhttp://www.wikidata.org/prop/di

In [7]:
lst2 = sorted([25797,25068,30,459,961,1335,2710,2106,87,444,1956,2119,10553,18985,12015])
print(lst2)
p_lst(lst2)

[30, 87, 444, 459, 961, 1335, 1956, 2106, 2119, 2710, 10553, 12015, 18985, 25068, 25797]
+++++++++++++++++++++++TOTAL+++++++++++++++++++++++++
15

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++queries2_30++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-------triple-------
1.0
-------bgp----------
1.0
------unions---------
0
-----optionals--------
0
-----selects------
1
------query--------
SELECT  ?var1 ?var1Label WHERE   { ?var1  <http://www.wikidata.org/prop/direct/P698>  11520265 } 
------trees---------
["VAR_URI_LITERALᶲhttp://www.wikidata.org/prop/direct/P698"]
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++queries2_87++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-------triple-------
1.0
-------bgp----------
1.0
------unions---------
0
-----optionals--------
0


["JOIN", [ "JOINᶲhttp://www.wikidata.org/prop/qualifier/value/P580ᶲhttp://www.wikidata.org/prop/qualifier/value/P582ᶲhttp://wikiba.se/ontology#timeValueᶲhttp://wikiba.se/ontology#timeValueᶲhttp://wikiba.se/ontology#timePrecisionᶲhttp://wikiba.se/ontology#timePrecision" ,   [ "JOINᶲhttp://www.wikidata.org/prop/qualifier/value/P580ᶲhttp://www.wikidata.org/prop/qualifier/value/P582ᶲhttp://wikiba.se/ontology#timeValueᶲhttp://wikiba.se/ontology#timeValueᶲhttp://wikiba.se/ontology#timePrecision" ,     [ "JOINᶲhttp://www.wikidata.org/prop/qualifier/value/P580ᶲhttp://www.wikidata.org/prop/qualifier/value/P582ᶲhttp://wikiba.se/ontology#timeValueᶲhttp://wikiba.se/ontology#timeValue" ,       [ "JOINᶲhttp://www.wikidata.org/prop/qualifier/value/P580ᶲhttp://www.wikidata.org/prop/qualifier/value/P582ᶲhttp://wikiba.se/ontology#timeValue" ,         [ "JOINᶲhttp://www.wikidata.org/prop/qualifier/value/P580ᶲhttp://www.wikidata.org/prop/qualifier/value/P582" ,           [ "VAR_URI_VARᶲhttp://www.wikidata

In [10]:
lst3 = [486, 929, 3570, 4878, 7428, 8811, 9691, 10874, 12036, 12245, 12463, 13248, 16640, 25390, 25515]
print(lst3)
p_lst(lst3)

[486, 929, 3570, 4878, 7428, 8811, 9691, 10874, 12036, 12245, 12463, 13248, 16640, 25390, 25515]
+++++++++++++++++++++++TOTAL+++++++++++++++++++++++++
15

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++queries2_486++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-------triple-------
0.0
-------bgp----------
0.0
------unions---------
0
-----optionals--------
0
-----selects------
1
------query--------
SELECT DISTINCT  ?var1 WHERE   { BIND(<http://www.wikidata.org/entity/Q12501> AS ?var2)     ?var2 ((((<http://www.wikidata.org/prop/direct/P30>|<http://www.wikidata.org/prop/direct/P17>)|<http://www.wikidata.org/prop/direct/P131>)|<http://www.wikidata.org/prop/direct/P376>)|<http://www.wikidata.org/prop/direct/P501>)* ?var1   } 
------trees---------
["URI_VAR_VARᶲhttp://www.wikidata.org/entity/Q18557119"]
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++

["LEFT_JOIN" , ["JOIN", ["JOIN", ["VAR_URI_URIᶲhttp://www.wikidata.org/prop/direct/P279"], ["VAR_URI_VARᶲhttp://www.wikidata.org/prop/direct/P279"]], [ "JOINᶲhttp://www.wikidata.org/prop/direct/P279ᶲhttp://www.wikidata.org/prop/direct/P17ᶲhttp://www.wikidata.org/prop/direct/P297" ,   [ "JOINᶲhttp://www.wikidata.org/prop/direct/P279ᶲhttp://www.wikidata.org/prop/direct/P17" ,     [ "VAR_URI_URIᶲhttp://www.wikidata.org/prop/direct/P279" ] ,     [ "VAR_URI_VARᶲhttp://www.wikidata.org/prop/direct/P17" ]   ] ,   [ "VAR_URI_LITERALᶲhttp://www.wikidata.org/prop/direct/P297" ] ] ] , ["VAR_URI_VARᶲhttp://www.wikidata.org/prop/direct/P360"]]
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++queries2_12245++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-------triple-------
1.0
-------bgp----------
1.0
------unions---------
0
-----optionals--------
1
-----selects------
1
------query--------
S